In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [5]:
df = pd.read_excel("dataset.xlsx")
df = df.loc[(df['Data'] >= '2022-01-01')]

df['Data_only'] = df['Data'].dt.date

df = df.dropna()
df.columns

# df.to_excel("datasetNA.xlsx")


Index(['Id', 'Titulo', 'Data', 'Localizacao', 'Categoria', 'Problema',
       'Produto', 'Descricao', 'Status', 'Resolvido', 'Avaliada',
       'VoltariaNegocio', 'Nota', 'Data_only'],
      dtype='object')

In [6]:
categoria = df['Categoria'].unique().tolist()

problema = df['Problema'].unique().tolist()
produto = df['Produto'].unique().tolist()
status = df['Status'].unique().tolist()
# resolvido = df['Resolvido'].unique().tolist()
# notaList = df['Nota'].unique().tolist()

allItens = categoria + problema + produto + status 

origin = []
originLabel = []
dest = []
destLabel = []
value = []
color = []
nodecolor = []


def insertItem (indexOrigin, indexDest, origin, dest, value, color):
    for index in range(len(origin)):
        if(origin[index] == indexOrigin):
               if(dest[index] == indexDest):
                    value[index]+= 1
                    return
    origin.append(indexOrigin)
    originLabel.append(allItens[indexOrigin])
    dest.append(indexDest)
    destLabel.append(allItens[indexDest])
    value.append(1) 

    colorItem = list(np.random.choice(range(256), size=3))
    color.append(f"rgba({colorItem[0]}, {colorItem[1]}, {colorItem[2]}, 0.8)")

def GenerateInterations (allItens, origin, dest, value, color):
    qtd = 1
    for index, row in df.iterrows():
        
        catIndex = allItens.index(row['Categoria'])
     
        probList =  allItens[len(categoria):]
        probIndex =  len(categoria) + probList.index(row['Problema'])
       
        if(probIndex < catIndex):
            print(f"Inversao de valores - {row['Categoria']} - {row['Problema']}" )
            # probIndex = allItens.index(row['Problema'])

        prodList = allItens[len(categoria) + len(problema):]
        prodIndex = len(categoria) + len(problema) + prodList.index(row['Produto'])
       
        if(prodIndex < probIndex):
            print(f"Inversao de valores - {row['Problema']} - {row['Produto']}" )
        
        startList = allItens[len(produto) + len(categoria) + len(problema):]
        statIndex = len(produto) + len(categoria) + len(problema) + startList.index(row['Status'])

        if(row['Problema'] == "Qualidade da internet"):
            if(row['Categoria'] == "Não encontrei meu problema"):
                # print(qtd)
                qtd = qtd+1
        insertItem(catIndex, probIndex, origin, dest, value, color)
        insertItem(probIndex, prodIndex, origin, dest, value, color)
        insertItem(prodIndex, statIndex, origin, dest, value, color)
       
    return origin, dest, value, color



origin, dest, value, color = GenerateInterations(allItens, origin, dest, value, color)

for i in range(len(origin)):
    # print(f"{origin[i]} - {dest[i]} : {value[i]}")
    print(f"{allItens[origin[i]]} - {allItens[dest[i]]} : {value[i]}")


for i in range(len(categoria)):
    nodecolor.append("blue")   

for i in range(len(problema)):
    nodecolor.append("red")

for i in range(len(produto)):
    nodecolor.append("yellow")

for i in range(len(status)):
    nodecolor.append("green")

Não encontrei meu problema - Qualidade da internet : 44
Qualidade da internet - Internet para casa : 325
Internet para casa - NOT_SOLVED : 239
Não encontrei meu problema - Cobrança indevida : 128
Cobrança indevida - Internet para casa : 345
Internet para casa - SOLVED : 823
Cobrança indevida - Pagamentos e Documentos : 7
Pagamentos e Documentos - SOLVED : 8
Não encontrei meu problema - Mau atendimento no SAC : 40
Mau atendimento no SAC - Canais de Atendimento : 33
Canais de Atendimento - SOLVED : 28
Provedores e serv. de internet - Cancelamento : 77
Cancelamento - Internet para casa : 106
Provedores e serv. de internet - Qualidade da internet : 306
Provedores e serv. de internet - Cobrança indevida : 262
Mau atendimento no SAC - Equipe de Atendimento : 18
Equipe de Atendimento - SOLVED : 17
Não encontrei meu problema - Propaganda enganosa : 36
Propaganda enganosa - Internet para casa : 79
Não encontrei meu problema - Problema com equipamento fornecido : 12
Problema com equipamento forn

In [4]:
fig = go.Figure(data=[go.Sankey(
    valueformat = "s",
    node = dict(
      pad = 20,
      thickness = 30,
      line = dict(color = "black", width = 2),
      label = allItens,
      color = nodecolor
    ),
    link = dict(
      source = origin, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = dest,
      value = value,
      color = color
  ))])

fig.update_layout(width=1500, height = 2500, font_size=13)
fig.write_html("Sankey.html")
# fig.show()